<a href="https://colab.research.google.com/github/AayushKamath/Coursera_Capstone/blob/master/Week5_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Data Science Capstone Project**

###**Problem Statement: Finding Neighboorhoods that are suitable locations for the opening of a new café**

###**Import Necessary Libraries**

In [131]:
pip install geocoder

In [132]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


####**Scraping data from Suburbs of Mumbai wikipedia page**

In [133]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text
soup = BeautifulSoup(data, 'html.parser')
N_List=[]

In [134]:
# Appending the neighborhood values in a list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    N_List.append(row.text)

In [135]:
# Converting the list into a dataframe
mumbai_df = pd.DataFrame({"Neighborhood": N_List})

mumbai_df

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup
5,Borivali
6,Charkop
7,Chembur
8,Dahisar
9,Devipada


###**Dropping rows for which the geo co-ordinates were not available**

In [136]:
mumbai_df.drop(mumbai_df.index[[11,20,24,40]], inplace=True)
mumbai_df.reset_index(drop=True, inplace=True)
mumbai_df.shape


(38, 1)

In [137]:
# Creating lists to store the latitude and longitude values of each neighborhood
latitude=[]
longitude=[]
for i in range(len(mumbai_df)) : 
  address = '{},Mumbai'.format((mumbai_df.iloc[i, 0]))
  geolocator = Nominatim(user_agent="explorer")
  location = geolocator.geocode(address)
  lat=location.latitude
  lng=location.longitude
  latitude.append(lat)
  longitude.append(lng)
  
# Merging the lists into a single Dataframe
df = pd.DataFrame(list(zip(latitude, longitude)), 
               columns =['Latitude', 'Longitude'])
df.head()
  

,Latitude,Longitude
0,19.119698,72.846420
1,19.039578,72.922156
2,19.061894,72.924792
3,19.054979,72.840220
4,19.143868,72.938433


In [138]:
# Updating our dataframe containing neighboorhood names by adding their respective latitude and longitude values
mumbai_df['Latitude'] = df['Latitude']
mumbai_df['Longitude'] = df['Longitude']
print(mumbai_df.shape)
mumbai_df.head()

(38, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.119698,72.846420
1,Anushakti Nagar,19.039578,72.922156
2,Baiganwadi,19.061894,72.924792
3,Bandra,19.054979,72.840220
4,Bhandup,19.143868,72.938433


In [139]:
# Saving the dataframe
mumbai_df.to_csv("mumbai_df.csv", index=False)

In [140]:
# Finding the geographical co-ordinate of Mumbai as it would act as the centre of our map
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="mumbai_explorer")
location = geolocator.geocode(address)
M_latitude = location.latitude
M_longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(M_latitude, M_longitude))

The geograpical coordinate of Mumbai are 19.0759899, 72.8773928.


###**Creating a map of Mumbai with key neighborhoods being marked**

In [141]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[M_latitude, M_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mumbai)  
    
map_mumbai

In [142]:
# Saving the map
map_mumbai.save('map_mumbai.html')

###**Foursquare Credentials**

In [166]:
CLIENT_ID = 'Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'Foursquare Secret' # your Foursquare Secret
VERSION = '20200722' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: Foursquare ID
CLIENT_SECRET:Foursquare Secret


###**Getting the top 100 venues in a 1000 meter radius for each neighborhood**

In [144]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [145]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1116, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.119698,72.84642,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.119698,72.84642,Narayan Sandwich,19.121398,72.850270,Sandwich Place
2,Andheri,19.119698,72.84642,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
3,Andheri,19.119698,72.84642,Cafe Alfa,19.119667,72.843560,Indian Restaurant
4,Andheri,19.119698,72.84642,Shawarma Factory,19.124591,72.840398,Falafel Restaurant


In [146]:
# Printing the total number of unique venue categories
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

# Displaying the first 25 unique venue categories
venues_df['VenueCategory'].unique()[:25]

There are 164 uniques categories.


array(['Bakery', 'Sandwich Place', 'Indian Restaurant',
       'Falafel Restaurant', 'Ice Cream Shop', 'Fast Food Restaurant',
       'Department Store', 'Pizza Place', 'Vegetarian / Vegan Restaurant',
       'Restaurant', 'Food Court', 'Platform', 'Food Truck', 'Hotel',
       'Chinese Restaurant', 'Café', 'Bowling Alley', 'Camera Store',
       'Accessories Store', 'Athletics & Sports', 'Coffee Shop',
       'Electronics Store', 'Smoke Shop', 'Food', 'Bus Station'],
      dtype=object)

###**Analyzing Each Neighborhood**

In [149]:
mumbai_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Office,Opera House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


####**Grouping rows by neighborhood and taking the mean of the frequency of occurrence of each category**

In [150]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhoods').mean().reset_index()
mumbai_grouped

,Neighborhoods,ATM,Accessories Store,Adult Boutique,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Office,Opera House,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pub,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Shop,Women's Store,Yoga Studio
0,Andheri,0.000000,0.030303,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.060606,0.030303,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.060606,0.000000,0.000000,0.0000,0.000000,0.030303,0.030303,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.060606,0.000000,0.090909,0.151515,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.030303,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.060606,0.000000,0.00000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000
1,Anushakti Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000

In [154]:
# Finding the frequency of cafes for all neighborhoods
print(len(mumbai_grouped[mumbai_grouped["Café"] > 0]))
mumbai_cafe = mumbai_grouped[["Neighborhoods","Café"]]
mumbai_cafe.head()


28


,Neighborhoods,Café
0,Andheri,0.060606
1,Anushakti Nagar,0.200000
2,Baiganwadi,0.000000
3,Bandra,0.086957
4,Bhandup,0.050000


###**Cluster Neighborhoods**

In [155]:
# set number of clusters
kclusters = 3

mumbai_clustering = mumbai_cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 2, 1, 1, 2, 1, 1, 1, 1], dtype=int32)

In [156]:
mumbai_merged = mumbai_cafe.copy()

# add clustering labels
mumbai_merged["Cluster Labels"] = kmeans.labels_
mumbai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mumbai_merged.head()

,Neighborhood,Café,Cluster Labels
0,Andheri,0.060606,1
1,Anushakti Nagar,0.200000,0
2,Baiganwadi,0.000000,2
3,Bandra,0.086957,1
4,Bhandup,0.050000,1


In [157]:
# merging dataframes to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(mumbai_df.set_index("Neighborhood"), on="Neighborhood")

print(mumbai_merged.shape)
mumbai_merged.head()

(38, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,Andheri,0.060606,1,19.119698,72.846420
1,Anushakti Nagar,0.200000,0,19.039578,72.922156
2,Baiganwadi,0.000000,2,19.061894,72.924792
3,Bandra,0.086957,1,19.054979,72.840220
4,Bhandup,0.050000,1,19.143868,72.938433


In [158]:
# sort the results by Cluster Labels
print(mumbai_merged.shape)
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)
mumbai_merged

(38, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
37,Worli,0.150000,0,19.011696,72.818070
1,Anushakti Nagar,0.200000,0,19.039578,72.922156
35,Vikhroli,0.222222,0,19.111480,72.928021
16,Kalyan,0.242424,0,19.137892,72.810668
31,Thakur village,0.081081,1,19.209719,72.875925
30,"Sonapur, Bhandup",0.086957,1,19.161420,72.937532
28,Shil Phata,0.045455,1,19.112779,73.009472
27,Seven Bungalows,0.101449,1,19.129762,72.821378
24,Mulund,0.054054,1,19.172290,72.956469
19,Kurla,0.076923,1,19.075990,72.877393


In [165]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters



In [161]:
# Places belonging to cluster 0
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
37,Worli,0.150000,0,19.011696,72.818070
1,Anushakti Nagar,0.200000,0,19.039578,72.922156
35,Vikhroli,0.222222,0,19.111480,72.928021
16,Kalyan,0.242424,0,19.137892,72.810668


In [162]:
# Places belonging to cluster 1
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
31,Thakur village,0.081081,1,19.209719,72.875925
30,"Sonapur, Bhandup",0.086957,1,19.161420,72.937532
28,Shil Phata,0.045455,1,19.112779,73.009472
27,Seven Bungalows,0.101449,1,19.129762,72.821378
24,Mulund,0.054054,1,19.172290,72.956469
19,Kurla,0.076923,1,19.075990,72.877393
17,Kandivali,0.086957,1,19.204159,72.851682
15,Juhu,0.091954,1,19.107021,72.827528
13,Grant Road,0.062500,1,18.964447,72.813573
0,Andheri,0.060606,1,19.119698,72.846420


In [163]:
# Places belonging to cluster 2
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
2,Baiganwadi,0.000000,2,19.061894,72.924792
34,Vashi,0.030303,2,19.075713,73.000354
33,Uttan,0.000000,2,19.253319,72.790515
32,Tilak Nagar (Mumbai),0.042553,2,19.069238,72.897846
29,"Sion, Mumbai",0.034483,2,19.046521,72.863283
5,Borivali,0.020000,2,19.229068,72.857363
14,Jogeshwari,0.000000,2,19.134899,72.848820
12,Goregaon,0.000000,2,19.164753,72.850018
23,Mogra Village,0.000000,2,19.128868,72.860822
22,Mira Road,0.036585,2,19.187896,72.836596


###**Conclusion**

The neighborhoods were divided into 3 clusters based on their frequency value for cafes. Neighborhoods in Cluster 0 show the highest frequency of cafes. For a person looking to open a new cafe, neighboorhoods belonging to cluster 0 might not be the best choice for business as there are already a lot of cafes in those neighborhoods. Cluster 1 shows neighborhoods with moderate frequency of cafes. Opening a cafe in these neighborhoods can be considered as the market isn't saturated and at the same time there seems to be a decent amount of demand for cafes. Cluster 2 has neighboorhoods with either low frequency of cafes or zero cafes. Opening a cafe in any one of these neighboorhood presents a high risk high reward scenario. On one hand, a cafe might be considered as a new experience for the residents which could lead to good revenue. Whereas, on the other hand, people might simply not react to the opening of a cafe in their neighborhood as it might not be considered as a necessity by the residents of the neighborhood in question.

This concludes the assignment.